In [2]:
from datasets import load_dataset
import pandas as pd


/home/mohammed/work/poc/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset('shreyanshu09/Block_Diagram', split='train', cache_dir='./datasets')

df = dataset.to_pandas()

Generating validation split: 100%|██████████| 8662/8662 [00:00<00:00, 12990.09 examples/s]


In [4]:
df

,id,image,ground_truth
0,Break (77),{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"{""gt_parse"": {""c2t"": ""Start is connected with ..."
1,Break (78),{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"{""gt_parse"": {""c2t"": ""Start is connected with ..."
2,Break (79),{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"{""gt_parse"": {""c2t"": ""Begin is conected with C..."
3,Break (80),{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"{""gt_parse"": {""c2t"": ""Begin is conected with S..."
4,Break (81),{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"{""gt_parse"": {""c2t"": ""Example is connected wit..."
...,...,...,...
76258,kor_real_world_95,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"{""gt_parse"": {""c2t"": ""프로세스 A는 디스크에 요구를 합니다. 디스..."
76259,kor_real_world_96,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"{""gt_parse"": {""c2t"": ""Start는 시작점이고 다음 문장을 출력합니..."
76260,kor_real_world_97,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"{""gt_parse"": {""c2t"": ""시작점에서 시작하여, 사전측정 단계로 이동합..."
76261,kor_real_world_98,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,"{""gt_parse"": {""c2t"": ""생과일주스 만들기 시작에서 시작합니다.생과일..."


In [5]:
filter_words = ['Break', 'Connect', 'Normal']
filtered_df = df[df["id"].str.contains("|".join(filter_words), regex=True, na=False)]


In [11]:
filtered_df['image'][0]

{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xe7\x00\x00\x02s\x08\x06\x00\x00\x005\x12\xdb\x9c\x00\x00\x00\x04gAMA\x00\x00\xb1\x8f\x0b\xfca\x05\x00\x00\x00 cHRM\x00\x00z&\x00\x00\x80\x84\x00\x00\xfa\x00\x00\x00\x80\xe8\x00\x00u0\x00\x00\xea`\x00\x00:\x98\x00\x00\x17p\x9c\xbaQ<\x00\x00\x00\tpHYs\x00\x00\x0b\x11\x00\x00\x0b\x11\x01\x7fd_\x91\x00\x00\x00\x06bKGD\x00\xff\x00\xff\x00\xff\xa0\xbd\xa7\x93\x00\x00\x00%tEXtdate:create\x002022-02-06T11:43:21+03:00\xe8y\x932\x00\x00\x00%tEXtdate:modify\x002022-02-06T11:43:21+03:00\x99$+\x8e\x00\x00\xad\xc1IDATx^\xed\x9d\x07\x9c]E\xd9\x87\'\xbd\xf7N\x02\t\x90\x84\x9e\x00\xd2\xa4J\xf9>$\x14Q\x04\x01E\xb1\x01\x8a\n~\xd8\x0b("(M@,D@\x11D\x04AA t\x81\xd0k \xa1\xa4\x90\x90\x04\x02\t\xe9\xbd\x91|\xe7y\xb3\xef:\\o\xd9\xdd\xbbw\xef\xb9\xf7\xfe\x9f\xfd\xcd\x9e>g\xce\xb9s\xe6?\xef;s\xe6\xb4\xda\x98\x10\x84\x10B\x08\x91\x1aZ\xd7M\x85\x10B\x08\x91\x12$\xceB\x08!D\xca\x908\x0b!\x84\x10)C\xe2,\x84\x10B\xa4\x0c\x89\xb3\x10B\x08\x912$\xceB\x08!D\xc

In [7]:
import os
from PIL import Image
import io

In [ ]:
os.makedirs('images', exist_ok=True)

metadata = []
offset = 500
for index, row in filtered_df.iterrows():
    
        image_bytes = row["image"]['bytes'] 
        label = row["ground_truth"] 
        
        # Convert bytes to image
        image = Image.open(io.BytesIO(image_bytes))
        
        # Save image with index as filename
        filename = f"{offset+index}.png"
        image.save(os.path.join('images', filename))

        # Append to metadata
        metadata.append((filename, label))

# Save metadata to CSV
metadata_df = pd.DataFrame(metadata, columns=["filename", "label"])
metadata_df.to_csv('metadata_filtered', index=False)

print(f"Saved {len(filtered_df)} images and metadata!")

Saved 396 images and metadata!


In [31]:
metadata = pd.read_csv('metadata.csv')

In [32]:
metadata

,filename,label
0,500.png,Start is connected with Check needs for projec...
1,501.png,Start is connected with Check software. If Che...
2,502.png,Begin is conected with Check your PC configura...
3,503.png,Begin is conected with SUM=0 which is then con...
4,504.png,Example is connected with Does the computer tu...
...,...,...
381,891.png,New Budget is connected with Make an edit whic...
382,892.png,Input Source is connected with Input Transduce...
383,893.png,Start is connected with Requirements which is ...
384,894.png,Start is connected with Power On which is then...


In [33]:
files = set(os.listdir('images'))

In [34]:
len(files)

53

In [35]:
filetered_metadata = metadata[metadata['filename'].isin(files)]

In [36]:
filetered_metadata

,filename,label
20,520.png,Begin is connected with READ num1 which is the...
21,521.png,Start is connected with Open Email which is th...
25,525.png,If Log In is OpenID login then Logged In and i...
42,542.png,A is connected with Process step 3 which is th...
48,548.png,Start is connected with Make decision. If Make...
56,556.png,Begin is connecetd with Are you happy? which i...
62,562.png,Start is connected with document-filter which ...
63,563.png,Start is connected with Log in which is connec...
65,565.png,Start is connected with Initialize Variable wh...
69,569.png,User is connected with Selection of problem ar...


In [37]:
filetered_metadata.to_csv('metadata_filtered.csv', index=False)

In [24]:
filetered_metadata['label']['gt_parse']

KeyError: 'gt_parse'

In [ ]:
import json
c2t = json.loads(filetered_metadata['label'][])

KeyError: 'gt_parse'

In [30]:
# os.makedirs('images', exist_ok=True)
import json
metadata = []
offset = 500
for index, row in filtered_df.iterrows():
    try:
        label = json.loads(row["ground_truth"])['gt_parse']['c2t']        # Assuming column name is 'label'
        
        # Convert bytes to image
        
        # Save image with index as filename
        filename = f"{offset+index}.png"
        # image.save(os. path.join('images', filename))
        label.replace('"', '\\"')
        # Append to metadata
        metadata.append((filename, label))
    except json.JSONDecodeError as e:
        print(e)
        print(row["ground_truth"])
        continue    
# Save metadata to CSV
metadata_df = pd.DataFrame(metadata, columns=["filename", "label"])
metadata_df.to_csv('metadata_filtered', index=False)

print(f"Saved {len(filtered_df)} images and metadata!")

Expecting ',' delimiter: line 1 column 91 (char 90)
{"gt_parse": {"c2t": "Start is connected with Scan Bar code which is then connected with "Reader" decrypts message and sends it to "3D object management interface" which is further connected with Check the website. Check the website is connected with The back-end server checks data and compares it with "3D objects database" which is then connected with Does data match the detail in "3D obects database". If Does data match the detail in "3D obects database" is No? then An error message is displayed on the user screen and if Does data match the detail in "3D obects database" is Yes? then The 3D object is displayed on the user screen."}}
Expecting ',' delimiter: line 1 column 114 (char 113)
{"gt_parse": {"c2t": "Start is connected with Username acceptable?. If Username acceptable? is No then Close as "Apolicy" or "Invalid" and If Username acceptable? is Yes then Any blocks?. If Any blocks? is No then Account already exist? which if Accou